In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import medmnist
from medmnist import INFO

In [3]:
data_flag= 'chestmnist'
download = True
info = INFO[data_flag]
DataClass = getattr(medmnist, info['python_class'])

data_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])


# Caricamento dei dataset
train_dataset = DataClass(split='train', transform=data_transform, download=download)
test_dataset = DataClass(split='test', transform=data_transform, download=download)

# DataLoader: organizza i dati in batch per l'addestramento
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

print(f"Dataset '{data_flag}' caricato.")
print(f"Dimensione training set: {len(train_dataset)}")
print(f"Dimensione test set: {len(test_dataset)}")

Dataset 'chestmnist' caricato.
Dimensione training set: 78468
Dimensione test set: 22433


In [9]:
# --- 2. Definizione del Modello Lineare Semplice ---

class SimpleLinearClassifier(nn.Module):
    def __init__(self):
        super(SimpleLinearClassifier, self).__init__()
        self.linear = nn.Linear(224 * 224, 1) # Input 224*224=50176, Output 1 (score)

    def forward(self, x):
        x = x.view(-1, 224 * 224) # Appiattisci l'immagine
        x = self.linear(x)
        return x

model = SimpleLinearClassifier()

# Sposta il modello sulla GPU se disponibile, altrimenti sulla CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Modello spostato su: {device}")

# --- 3. Configurazione Addestramento ---

# Calcolo del pos_weight per gestire lo sbilanciamento delle classi
# Il numero di esempi positivi e negativi nel training set
num_positives = (train_dataset.labels[:, 1] == 1).sum().item()
num_negatives = (train_dataset.labels[:, 1] == 0).sum().item()

# pos_weight = Numero di campioni negativi / Numero di campioni positivi
pos_weight = torch.tensor(num_negatives / num_positives, dtype=torch.float32).to(device)

print(f"\nNumero di esempi positivi nel training set: {num_positives}")
print(f"Numero di esempi negativi nel training set: {num_negatives}")
print(f"Positional weight per BCEWithLogitsLoss: {pos_weight.item():.2f}")


criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight) # Funzione di costo per classificazione binaria
optimizer = optim.SGD(model.parameters(), lr=0.01) # Ottimizzatore
num_epochs = 10 # Numero di epoche di addestramento (puoi aumentarlo)

# --- 4. Addestramento del Modello ---

print("\nInizio addestramento del modello...")
for epoch in range(num_epochs):
    model.train() # Imposta il modello in modalità training
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels[:, 1].float().to(device).unsqueeze(1)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoca [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

print("Addestramento completato.")

Modello spostato su: cpu

Numero di esempi positivi nel training set: 1950
Numero di esempi negativi nel training set: 76518
Positional weight per BCEWithLogitsLoss: 39.24

Inizio addestramento del modello...
Epoca [1/10], Loss: 23.6070
Epoca [2/10], Loss: 21.0779
Epoca [3/10], Loss: 21.6470
Epoca [4/10], Loss: 20.9092
Epoca [5/10], Loss: 20.9376
Epoca [6/10], Loss: 20.3405
Epoca [7/10], Loss: 22.2467
Epoca [8/10], Loss: 20.7951
Epoca [9/10], Loss: 24.2209
Epoca [10/10], Loss: 20.8261
Addestramento completato.


In [10]:
# --- 5. Valutazione del Modello sul Test Set ---
print("\nValutazione del modello sul test set...")
model.eval() # Imposta il modello in modalità valutazione
correct = 0
total = 0
true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0

with torch.no_grad(): # Disabilita il calcolo dei gradienti per l'inferenza
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels[:, 1].to(device).unsqueeze(1)

        outputs = model(images)
        # Applica sigmoid e soglia di 0.5 per le previsioni binarie
        predicted = torch.sigmoid(outputs).round().squeeze()

        total += labels.size(0)
        correct += (predicted == labels.squeeze()).sum().item() # `squeeze()` rimuove la dimensione di 1 da (batch_size, 1)


        predicted_np = predicted.cpu().numpy()
        labels_np = labels.cpu().numpy().squeeze()

        true_positives += ((predicted_np == 1) & (labels_np == 1)).sum()
        true_negatives += ((predicted_np == 0) & (labels_np == 0)).sum()
        false_positives += ((predicted_np == 1) & (labels_np == 0)).sum()
        false_negatives += ((predicted_np == 0) & (labels_np == 1)).sum()

accuracy = 100 * correct / total
sensitivity = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
specificity = true_negatives / (true_negatives + false_positives) if (true_negatives + false_positives) > 0 else 0
precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0

print(f"Accuracy sul test set: {accuracy:.2f}%")
print("\nMatrice di Confusione:")
print(f"True Positives (TP): {true_positives}")
print(f"True Negatives (TN): {true_negatives}")
print(f"False Positives (FP): {false_positives}")
print(f"False Negatives (FN): {false_negatives}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Precision: {precision:.4f}")


Valutazione del modello sul test set...
Accuracy sul test set: 88.07%

Matrice di Confusione:
True Positives (TP): 185
True Negatives (TN): 19572
False Positives (FP): 2279
False Negatives (FN): 397
Sensitivity (Recall): 0.3179
Specificity: 0.8957
Precision: 0.0751


In [11]:
# --- 6. Estrazione e Salvataggio dei Pesi ---

# Accedi ai pesi dello strato lineare.
# `.data.cpu().numpy()` estrae i pesi come array NumPy dalla GPU (se usata) alla CPU.
# `.squeeze()` rimuove eventuali dimensioni singole (es. da (1, 50176) a (50176,)).
trained_weights = model.linear.weight.data.cpu().numpy().squeeze()

# Lo ridimensioniamo a 224x224 per coerenza con il tuo vecchio approccio e per la visualizzazione.
# Il tuo C++ li leggerà come un array 1D piatto, quindi la forma 2D è più per la nostra comodità.
trained_weights_reshaped = trained_weights.reshape((224, 224))
print(f"\nPesi estratti con forma appiattita: {trained_weights.shape}, ridimensionati a {trained_weights_reshaped.shape}")

# Percorso dove salvare i pesi addestrati
output_weights_dir = '../data/weights' # Utilizza il percorso relativo che avevi
os.makedirs(output_weights_dir, exist_ok=True)
weights_output_filepath = os.path.join(output_weights_dir, 'cardiomegaly_weights_224x224_trained.npy')

np.save(weights_output_filepath, trained_weights_reshaped)
print(f"Pesi addestrati salvati in: {weights_output_filepath}")

# Se vuoi visualizzare i pesi addestrati, puoi aggiungere la tua vecchia cella di visualizzazione qui.
# plt.figure(figsize=(6, 6))
# plt.imshow(trained_weights_reshaped, cmap='hot', origin='upper')
# plt.title('Mappa dei Pesi Addestrati per la Cardiomegalia')
# plt.colorbar(label='Peso')
# plt.show()


Pesi estratti con forma appiattita: (50176,), ridimensionati a (224, 224)
Pesi addestrati salvati in: ../data/weights/cardiomegaly_weights_224x224_trained.npy
